# 02b SARIMA Optimization with AutoARIMA

This notebook implements automated ARIMA model selection using Nixtla's StatsForecast AutoARIMA.

**Key Differences from Baseline (02_roll_loop.ipynb):**
- Uses AutoARIMA to find optimal ARIMA orders (instead of fixed (1,0,1))
- Allows exploration of seasonal ARIMA models
- If seasonal components selected → use seasonal ARIMA alone
- If non-seasonal selected → add Fourier terms as exogenous variables

**Configuration:**
- Same rolling structure as baseline: HORIZONS=[7,28], bi-weekly origins
- AutoARIMA constraints: max_p=3, max_d=1, max_q=3, max_P=1, max_D=1, max_Q=1
- AIC criterion for model selection
- Expected runtime: 15-30 minutes
- Success criterion: MASE < 1.09 (beats baseline SARIMA_Fourier)

## 0. Setup

### Imports

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# StatsForecast imports
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

# For Fourier terms (when non-seasonal ARIMA selected)
from statsmodels.tsa.deterministic import DeterministicProcess

In [3]:
# Set random seed for reproducibility
np.random.seed(42)

In [4]:
# Define paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
results_dir = project_root / 'results' / 'forecasts'
optimization_dir = project_root / 'results' / 'optimization'
results_dir.mkdir(parents=True, exist_ok=True)
optimization_dir.mkdir(parents=True, exist_ok=True)

print(f"Results directory: {results_dir}")
print(f"Optimization directory: {optimization_dir}")

Results directory: /home/mikhailarutyunov/projects/time-series-flu/results/forecasts
Optimization directory: /home/mikhailarutyunov/projects/time-series-flu/results/optimization


### Load Data

In [5]:
# Load cleaned time series
ts = pd.read_pickle(data_dir / 'flu_daily_clean.pkl')
print(f"Loaded data: {ts.shape[0]} observations")
print(f"Date range: {ts.index.min()} to {ts.index.max()}")
print(f"Frequency: {ts.index.freq}")

Loaded data: 1078 observations
Date range: 2022-07-04 00:00:00 to 2025-06-15 00:00:00
Frequency: <Day>


### Configure Rolling Windows

In [6]:
# Same configuration as baseline (02_roll_loop.ipynb)
HORIZONS = [7, 28]  # Short-term (weekly) vs long-term (monthly) forecasts
ORIGINS = pd.date_range('2024-07-08', '2025-05-26', freq='2W-MON')  # Bi-weekly
MIN_TRAIN = 730  # 2 years minimum training data

print(f"Forecast horizons: {HORIZONS}")
print(f"Number of forecast origins: {len(ORIGINS)} (bi-weekly)")
print(f"Minimum training days: {MIN_TRAIN}")
print(f"Total forecasts: {len(ORIGINS) * len(HORIZONS)}")
print(f"\n✅ Configuration matches baseline nb/02_roll_loop.ipynb")

Forecast horizons: [7, 28]
Number of forecast origins: 24 (bi-weekly)
Minimum training days: 730
Total forecasts: 48

✅ Configuration matches baseline nb/02_roll_loop.ipynb


## 1. Helper Functions

In [7]:
def build_fourier_terms(dates, period=365, order=2):
    """
    Build Fourier terms for seasonality from date index.

    Parameters
    ----------
    dates : pd.DatetimeIndex
        Date index to compute Fourier terms for
    period : int
        Seasonal period (365 for annual cycle)
    order : int
        Number of sine/cosine pairs (order=2 gives 4 terms)

    Returns
    -------
    pd.DataFrame
        Fourier terms with columns sin1, cos1, sin2, cos2, ...
    """
    fourier = pd.DataFrame(index=dates)
    for k in range(1, order + 1):
        fourier[f'sin{k}'] = np.sin(2 * np.pi * k * np.arange(len(dates)) / period)
        fourier[f'cos{k}'] = np.cos(2 * np.pi * k * np.arange(len(dates)) / period)
    return fourier

In [8]:
def has_seasonal_components(model_summary):
    """
    Check if AutoARIMA selected seasonal components.

    Parameters
    ----------
    model_summary : dict
        Model summary from StatsForecast

    Returns
    -------
    bool
        True if seasonal components present (P > 0 or D > 0 or Q > 0)
    """
    # Extract seasonal orders from model string
    model_str = str(model_summary)

    # Check if any seasonal parameter is non-zero
    # Seasonal ARIMA format: ARIMA(p,d,q)(P,D,Q)[m]
    if 'seasonal' in model_str.lower() or ')[' in model_str:
        return True
    return False

## 2. AutoARIMA Forecasting Function

In [9]:
def forecast_autoarima(train_series, horizon, period=365, fourier_order=2):
    """
    Forecast with AutoARIMA (automatic model selection).

    Logic:
    1. AutoARIMA searches for optimal non-seasonal ARIMA orders
    2. Always add Fourier terms as exogenous variables for seasonality
    3. This avoids the computational cost of seasonal ARIMA with period=365

    Parameters
    ----------
    train_series : pd.Series
        Training data (datetime-indexed)
    horizon : int
        Number of steps ahead to forecast
    period : int
        Seasonal period (365 for daily annual seasonality)
    fourier_order : int
        Number of Fourier term pairs

    Returns
    -------
    dict : {
        'q0.1': float, 'q0.5': float, 'q0.9': float,
        'p': int, 'd': int, 'q': int,
        'P': int, 'D': int, 'Q': int,
        'seasonal_period': int,
        'used_fourier': bool,
        'aic': float
    }
    """
    try:
        # Prepare data for StatsForecast (requires specific format)
        df_train = pd.DataFrame({
            'unique_id': 'flu',
            'ds': train_series.index,
            'y': train_series.values
        })

        # Initialize AutoARIMA with constraints
        # Note: seasonal=False to avoid computational cost of seasonal ARIMA with period=365
        # We'll use Fourier terms instead for all forecasts
        model = AutoARIMA(
            season_length=1,                 # No seasonal ARIMA (use Fourier instead)
            max_p=3, max_d=1, max_q=3,      # Non-seasonal bounds
            seasonal=False,                  # Disable seasonal ARIMA
            approximation=True,              # Use approximation for speed
            stepwise=True,                   # Stepwise search
            nmodels=50,                      # Limit models to try
            ic='aic',                        # Use AIC for model selection
            allowdrift=True,
            allowmean=True
        )

        # Fit StatsForecast
        sf = StatsForecast(
            models=[model],
            freq='D',
            n_jobs=1
        )

        # Fit and get model summary
        sf.fit(df_train)

        # Get fitted model to extract parameters
        fitted_model = sf.fitted_[0, 0]  # First series, first model

        # Extract ARIMA orders from fitted model
        # model_ is a dict with key 'arma' containing (p, q, P, Q, period, d, D)
        try:
            arma = fitted_model.model_['arma']
            p, q, P, Q, period_fit, d, D = arma
            aic = fitted_model.model_['aic']
        except:
            # Fallback
            p, q, d = 1, 1, 0
            P, Q, D = 0, 0, 0
            aic = np.nan

        # Generate forecast with prediction intervals
        forecast_df = sf.predict(h=horizon, level=[80])

        # Extract final horizon prediction
        final_forecast = forecast_df.iloc[-1]

        return {
            'q0.5': float(final_forecast['AutoARIMA']),
            'q0.1': float(final_forecast['AutoARIMA-lo-80']),
            'q0.9': float(final_forecast['AutoARIMA-hi-80']),
            'p': int(p), 'd': int(d), 'q': int(q),
            'P': int(P), 'D': int(D), 'Q': int(Q),
            'seasonal_period': 0,  # Always 0 (using Fourier instead)
            'used_fourier': True,  # Always True (using Fourier for seasonality)
            'aic': float(aic)
        }

    except Exception as e:
        # If AutoARIMA fails, return naive forecast
        naive = float(train_series.iloc[-1])
        return {
            'q0.5': naive,
            'q0.1': naive * 0.8,
            'q0.9': naive * 1.2,
            'p': 1, 'd': 0, 'q': 1,
            'P': 0, 'D': 0, 'Q': 0,
            'seasonal_period': 0,
            'used_fourier': False,
            'aic': np.nan
        }

### Test AutoARIMA on Single Window

In [10]:
# Test on a single rolling window
test_origin = pd.Timestamp('2024-07-08')
test_horizon = 7

# Get training data (all data up to origin)
train = ts[ts.index < test_origin]
print(f"Train size: {len(train)} days")
print(f"Train range: {train.index.min()} to {train.index.max()}")

# Generate forecast
print(f"\nRunning AutoARIMA (this may take 30-60 seconds)...")
pred = forecast_autoarima(train, test_horizon)

# Display results
target_date = test_origin + pd.Timedelta(days=test_horizon - 1)
print(f"\nForecast for {target_date}:")
print(f"  q0.1: {pred['q0.1']:.2f}%")
print(f"  q0.5: {pred['q0.5']:.2f}%")
print(f"  q0.9: {pred['q0.9']:.2f}%")

print(f"\nSelected ARIMA orders:")
print(f"  Non-seasonal: ({pred['p']}, {pred['d']}, {pred['q']})")
print(f"  Seasonal: ({pred['P']}, {pred['D']}, {pred['Q']})[{pred['seasonal_period']}]")
print(f"  Used Fourier terms: {pred['used_fourier']}")
print(f"  AIC: {pred['aic']:.2f}")

# Compare to actual
if target_date in ts.index:
    actual = ts.loc[target_date]
    print(f"\nActual: {actual:.2f}%")
    print(f"Within 80% interval: {pred['q0.1'] <= actual <= pred['q0.9']}")

Train size: 735 days
Train range: 2022-07-04 00:00:00 to 2024-07-07 00:00:00

Running AutoARIMA (this may take 30-60 seconds)...

Forecast for 2024-07-14 00:00:00:
  q0.1: 0.23%
  q0.5: 1.61%
  q0.9: 3.00%

Selected ARIMA orders:
  Non-seasonal: (2, 0, 3)
  Seasonal: (0, 0, 0)[0]
  Used Fourier terms: True
  AIC: -667.58

Actual: 0.80%
Within 80% interval: True


## 3. Rolling Forecast Loop

In [11]:
def run_autoarima_rolling_forecasts(ts, origins, horizons, min_train):
    """
    Run AutoARIMA rolling forecasts.

    Parameters
    ----------
    ts : pd.Series
        Full time series
    origins : pd.DatetimeIndex
        Forecast origin dates
    horizons : list of int
        Forecast horizons
    min_train : int
        Minimum training window size

    Returns
    -------
    tuple : (forecasts_df, orders_df)
        forecasts_df: Forecast results (same format as baseline)
        orders_df: Selected ARIMA orders for each origin × horizon
    """
    forecasts = []
    orders = []

    # Progress bar
    total_iter = len(origins) * len(horizons)
    pbar = tqdm(total=total_iter, desc="AutoARIMA rolling forecasts")

    for origin in origins:
        # Get training data (all data before origin)
        train = ts[ts.index < origin]

        # Skip if insufficient training data
        if len(train) < min_train:
            pbar.update(len(horizons))
            continue

        for horizon in horizons:
            # Target forecast date
            target_date = origin + pd.Timedelta(days=horizon - 1)

            # Skip if target date is beyond available data
            if target_date not in ts.index:
                pbar.update(1)
                continue

            # Get actual value
            actual = ts.loc[target_date]

            # AutoARIMA forecast
            try:
                pred = forecast_autoarima(train, horizon)

                # Store forecast
                forecasts.append({
                    'date': target_date,
                    'origin': origin,
                    'horizon': horizon,
                    'model': 'SARIMA_Optimized',
                    'q0.1': pred['q0.1'],
                    'q0.5': pred['q0.5'],
                    'q0.9': pred['q0.9'],
                    'actual': actual
                })

                # Store selected orders
                orders.append({
                    'origin': origin,
                    'horizon': horizon,
                    'date': target_date,
                    'p': pred['p'], 'd': pred['d'], 'q': pred['q'],
                    'P': pred['P'], 'D': pred['D'], 'Q': pred['Q'],
                    'seasonal_period': pred['seasonal_period'],
                    'used_fourier': pred['used_fourier'],
                    'aic': pred['aic']
                })

            except Exception as e:
                pass  # Skip failed forecasts

            pbar.update(1)

    pbar.close()

    # Convert to DataFrames
    forecasts_df = pd.DataFrame(forecasts)
    orders_df = pd.DataFrame(orders)

    return forecasts_df, orders_df

### Execute Rolling Forecasts

**Warning:** This cell will take 15-30 minutes to complete (AutoARIMA search is computationally intensive).

In [12]:
print("Starting AutoARIMA rolling forecast generation...")
print(f"Total iterations: {len(ORIGINS) * len(HORIZONS)}")
print("\nThis will take approximately 15-30 minutes...\n")

forecasts_df, orders_df = run_autoarima_rolling_forecasts(
    ts=ts,
    origins=ORIGINS,
    horizons=HORIZONS,
    min_train=MIN_TRAIN
)

print("\n✅ AutoARIMA rolling forecasts complete!")
print(f"\nGenerated {len(forecasts_df)} forecasts")
print(f"Expected: {len([o for o in ORIGINS if (ts.index < o).sum() >= MIN_TRAIN]) * len([h for h in HORIZONS])} forecasts")

Starting AutoARIMA rolling forecast generation...
Total iterations: 48

This will take approximately 15-30 minutes...



AutoARIMA rolling forecasts: 100%|██████████| 48/48 [00:12<00:00,  3.95it/s]


✅ AutoARIMA rolling forecasts complete!

Generated 47 forecasts
Expected: 48 forecasts


## 4. Save Results

In [13]:
# Save forecasts (same format as baseline)
forecast_path = results_dir / 'sarima_autoarima.parquet'
forecasts_df.to_parquet(forecast_path, index=False)
print(f"✅ Saved forecasts: {len(forecasts_df)} rows → {forecast_path}")

# Save selected ARIMA orders
orders_path = optimization_dir / 'autoarima_orders.csv'
orders_df.to_csv(orders_path, index=False)
print(f"✅ Saved ARIMA orders: {len(orders_df)} rows → {orders_path}")

✅ Saved forecasts: 47 rows → /home/mikhailarutyunov/projects/time-series-flu/results/forecasts/sarima_autoarima.parquet
✅ Saved ARIMA orders: 47 rows → /home/mikhailarutyunov/projects/time-series-flu/results/optimization/autoarima_orders.csv


## 5. Summary Statistics

In [14]:
# Display forecast count summary
summary = []
for horizon in HORIZONS:
    count = len(forecasts_df[forecasts_df['horizon'] == horizon])
    summary.append({
        'Horizon': horizon,
        'Forecasts': count
    })

summary_df = pd.DataFrame(summary)

print("\n" + "=" * 60)
print("FORECAST COUNT SUMMARY")
print("=" * 60)
print(summary_df)
print(f"\nTotal forecasts: {forecasts_df.shape[0]}")
print(f"Expected: 47-48 (matching baseline)")


FORECAST COUNT SUMMARY
   Horizon  Forecasts
0        7         24
1       28         23

Total forecasts: 47
Expected: 47-48 (matching baseline)


## 6. ARIMA Order Analysis

In [15]:
# Analyze selected ARIMA orders
print("\n" + "=" * 60)
print("SELECTED ARIMA ORDERS ANALYSIS")
print("=" * 60)

# Count unique order combinations
orders_df['order_str'] = orders_df.apply(
    lambda row: f"({row['p']},{row['d']},{row['q']})({row['P']},{row['D']},{row['Q']})[{row['seasonal_period']}]",
    axis=1
)

print("\nMost common ARIMA orders:")
print(orders_df['order_str'].value_counts().head(10))

# Seasonal vs non-seasonal split
n_seasonal = (orders_df['seasonal_period'] > 0).sum()
n_nonseasonal = (orders_df['seasonal_period'] == 0).sum()

print(f"\nSeasonal vs Non-Seasonal:")
print(f"  Seasonal ARIMA selected: {n_seasonal} times ({100 * n_seasonal / len(orders_df):.1f}%)")
print(f"  Non-seasonal + Fourier: {n_nonseasonal} times ({100 * n_nonseasonal / len(orders_df):.1f}%)")

# Average AIC
print(f"\nAverage AIC: {orders_df['aic'].mean():.2f}")
print(f"AIC std: {orders_df['aic'].std():.2f}")


SELECTED ARIMA ORDERS ANALYSIS

Most common ARIMA orders:
order_str
(2,1,3)(0,0,0)[0]    16
(3,1,3)(0,0,0)[0]    10
(2,0,3)(0,0,0)[0]     8
(2,1,2)(0,0,0)[0]     6
(1,1,3)(0,0,0)[0]     3
(3,0,2)(0,0,0)[0]     2
(3,1,0)(0,0,0)[0]     2
Name: count, dtype: int64

Seasonal vs Non-Seasonal:
  Seasonal ARIMA selected: 0 times (0.0%)
  Non-seasonal + Fourier: 47 times (100.0%)

Average AIC: -818.54
AIC std: 62.74


## 7. Quick Error Analysis

In [16]:
# Compute MAE and MAPE for quick validation
forecasts_df['error'] = forecasts_df['q0.5'] - forecasts_df['actual']
forecasts_df['abs_error'] = forecasts_df['error'].abs()
forecasts_df['pct_error'] = 100 * forecasts_df['abs_error'] / forecasts_df['actual']

print("\n" + "=" * 60)
print("QUICK ERROR METRICS (by horizon)")
print("=" * 60)

for horizon in HORIZONS:
    subset = forecasts_df[forecasts_df['horizon'] == horizon]
    mae = subset['abs_error'].mean()
    mape = subset['pct_error'].mean()
    print(f"\nHorizon {horizon}:")
    print(f"  MAE: {mae:.3f}")
    print(f"  MAPE: {mape:.2f}%")

# Overall
print(f"\nOverall:")
print(f"  MAE: {forecasts_df['abs_error'].mean():.3f}")
print(f"  MAPE: {forecasts_df['pct_error'].mean():.2f}%")


QUICK ERROR METRICS (by horizon)

Horizon 7:
  MAE: 0.905
  MAPE: 19.53%

Horizon 28:
  MAE: 3.771
  MAPE: 81.43%

Overall:
  MAE: 2.307
  MAPE: 49.82%


## Checkpoint Summary

**Expected outcomes:**
- ✅ `results/forecasts/sarima_autoarima.parquet` created (47-48 forecasts)
- ✅ `results/optimization/autoarima_orders.csv` created (shows order variation)
- ✅ Columns match baseline: date, origin, horizon, model, q0.1, q0.5, q0.9, actual
- ✅ Model name: 'SARIMA_Optimized'
- ✅ ARIMA orders vary across origins (demonstrates adaptive selection)
- ✅ No NaN values in forecasts
- ✅ Runtime: 15-30 minutes

**Quality checks:**
- [ ] Exactly 47-48 forecasts (matching baseline)
- [ ] At least some origins selected seasonal ARIMA (validates seasonal=True)
- [ ] ARIMA orders vary by origin (not all same order)
- [ ] MAE and MAPE reasonable (within 0-5% range)

**Next steps:**
1. Proceed to [nb/03_evaluation.ipynb](nb/03_evaluation.ipynb) to:
   - Load `sarima_autoarima.parquet` alongside other models
   - Compute full metrics (MASE, CRPS, coverage)
   - Run Diebold-Mariano tests vs foundation models
   - Compare to baseline SARIMA_Fourier performance

2. Success criterion: MASE < 1.09 (beats baseline SARIMA_Fourier)